# Word Vectors and the FNC

## 0. Import packages.

In [ ]:
import nltk
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

## 1. Download the word vectors.

Go to https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit and download GoogleNews-vectors-negative300.bin.gz. When it finishes downloadin, extract the file into this folder.

This may take awhile. While it's downloading, read this page for more information about Word2Vec: https://code.google.com/archive/p/word2vec/. Then, read this more in-depth description: https://www.tensorflow.org/tutorials/representation/word2vec

## 2. Load the word vectors.

In [ ]:
word2vec = {}

_wnl = nltk.WordNetLemmatizer()
def normalize_word(w):
    return _wnl.lemmatize(w).lower()

# Load Google's pre-trained Word2Vec model.
def initialize():
    global word2vec
    if len(word2vec) == 0:
        print('loading word2vec...')
        word_vectors = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        for word in word_vectors.vocab:
            word2vec[normalize_word(word)] = word_vectors[word]
        print('word2vec loaded')

In [ ]:
initialize()

Now, word2vec is a KeyedVector, and you can view the vectors corresponding to various words. If you're curious, try running the following cell with different words.

In [ ]:
word2vec['hi']

## 2. Write a function to convert a sentence into a vector.

The word vectors we downloaded give us a vector for each word, but we want a vector to represent each sentence. This function does just that. Read through the code, and try to answer the following questions:

**In Line $2$, why do we write `[0.0] * 300`?** (Hint: try running the following cell and see what happens).

In [ ]:
[0.0] * 5

**Why do we need the `if` statement on line $6$? What do you think would happen if we removed it?**

**What's the point of line $9$ and line $10$?** (Hint: Think about why we divided by the norms of the vectors when computing cosine similarity).

In [ ]:
def sentence2vector(sentence, word2vec):
    vector = np.array([0.0] * 300)
    count = 0
    for word in sentence:
        if word in word2vec:
            vector += word2vec[word]
            count += 1
    if count > 0:
        vector /= count
        vector /= np.linalg.norm(vector)
    return vector

Now, try creating vectors from sentences.

In [ ]:
title1 = "sam I am"
title2 = "I am sam"
title3 = "green eggs and ham"

In [ ]:
titlevector1 = sentence2vector(____, ____) # TODO: FILL IN HERE.
titlevector2 = sentence2vector(____, ____) # TODO: FILL IN HERE.
titlevector3 = sentence2vector(____, ____) # TODO: FILL IN HERE.

Compute the cosine similarities between these titles. Do the results surprise you?

In [ ]:
def compute_cosine_similarity(vector1, vector2):
    return np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))

In [ ]:
print(compute_cosine_similarity(titlevector1, titlevector2))
print(compute_cosine_similarity(titlevector1, titlevector3))

Try creating more sentence vectors of your own choosing.

In [ ]:
title_a = ____ # TODO: FILL IN HERE
title_B = ____ # TODO: FILL IN HERE
titlevector_a = sentence2vector(____, ____) # TODO: FILL IN HERE.
titlevector_b = sentence2vector(____, ____) # TODO: FILL IN HERE.

In [ ]:
print(compute_cosine_similarity(titlevector_a, titlevector_b))

## 3. Write a function to compute similarity using sentence vectors.

Instead of directly computing the cosine similarity between pairs of sentences, the FNC solution uses a slightly different similarity metric. This is one part of their semantic_similarities feature (we'll talk a bit more about the other part tomorrow, when we put everything together).

Read the code, and try to answer the following questions:

**Lines $7$ through $16$ loop through each sentence in `body_sentences`. What does this loop do with each body sentence?** (The `body_sentences` argument is a list of all the sentences in an article body.)

**What does the returned feature include?**

**Try to re-write this function in pseudo-code, writing a one-line description of what each line of code does.**

In [ ]:
def semantic_similarities(title, body_sentences, word2vec):
    title_vector = sentence2vector(title, word2vec)
    max_sim = -1
    best_vector = np.array([0.0] * 300)

    supports = []
    for sub_body in body_sentences:
        sub_body_vector = sentence2vector(sub_body, word2vec)
        similarity = 0
        for i in range(300):
            similarity += title_vector[i] * sub_body_vector[i]
        if similarity > max_sim:
            max_sim = similarity
            best_vector = sub_body_vector

        supports.append(similarity)

    features = [max(supports), min(supports)]

    for v in best_vector:
        features.append(v)
    for v in title_vector:
        features.append(v)
    return features

## We'll build upon this code tomorrow, when we start looking at the full FNC solution!